# AT PyTorch Demo — Deep Learning Fundamentals (V3)


In [ ]:
# Setup
import math
import random
import numpy as np
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader

SEED = 7
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


## Data


In [ ]:
def make_rings(n_per_class=500, r_inner=1.0, r_outer=3.0, noise=0.15):
    a1 = 2 * math.pi * np.random.rand(n_per_class)
    a2 = 2 * math.pi * np.random.rand(n_per_class)
    inner = np.c_[r_inner * np.cos(a1), r_inner * np.sin(a1)] + noise * np.random.randn(n_per_class, 2)
    outer = np.c_[r_outer * np.cos(a2), r_outer * np.sin(a2)] + noise * np.random.randn(n_per_class, 2)
    X = np.vstack([inner, outer]).astype(np.float32)
    y = np.r_[np.zeros(n_per_class), np.ones(n_per_class)].astype(np.int64)
    idx = np.random.permutation(len(X))
    return torch.from_numpy(X[idx]), torch.from_numpy(y[idx])


def make_spiral(n_per_class=500, turns=2.5, noise=0.3):
    n = n_per_class
    t = np.linspace(0.0, turns * math.pi, n)
    r = np.linspace(0.2, 1.0, n)
    x1 = r * np.cos(t) + noise * np.random.randn(n)
    y1 = r * np.sin(t) + noise * np.random.randn(n)
    x2 = r * np.cos(t + math.pi) + noise * np.random.randn(n)
    y2 = r * np.sin(t + math.pi) + noise * np.random.randn(n)
    X = np.vstack([np.c_[x1, y1], np.c_[x2, y2]]).astype(np.float32)
    y = np.r_[np.zeros(n), np.ones(n)].astype(np.int64)
    idx = np.random.permutation(len(X))
    return torch.from_numpy(X[idx]), torch.from_numpy(y[idx])


def split_loaders(X, y, batch_size=128, val_ratio=0.2):
    n = len(X)
    n_val = int(n * val_ratio)
    perm = torch.randperm(n)
    val_idx, train_idx = perm[:n_val], perm[n_val:]
    train_ds = TensorDataset(X[train_idx], y[train_idx])
    val_ds = TensorDataset(X[val_idx], y[val_idx])
    return (
        DataLoader(train_ds, batch_size=batch_size, shuffle=True),
        DataLoader(val_ds, batch_size=batch_size, shuffle=False),
    )


## Model and training


In [ ]:
class SimpleMLP(nn.Module):
    def __init__(self, width=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, width), nn.ReLU(),
            nn.Linear(width, width), nn.ReLU(),
            nn.Linear(width, 2),
        )

    def forward(self, x):
        return self.net(x)


def accuracy(logits, y):
    return (logits.argmax(1) == y).float().mean().item()


def train(model, train_loader, val_loader, epochs=200, lr=1e-3):
    model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.CrossEntropyLoss()

    history = {"train_loss": [], "val_acc": []}
    for epoch in range(epochs):
        model.train()
        running = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            opt.zero_grad()
            logits = model(xb)
            loss = loss_fn(logits, yb)
            loss.backward()
            opt.step()
            running += loss.item() * xb.size(0)
        train_loss = running / len(train_loader.dataset)

        model.eval()
        correct, n = 0, 0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(device), yb.to(device)
                logits = model(xb)
                correct += (logits.argmax(1) == yb).sum().item()
                n += xb.size(0)
        val_acc = correct / n
        history["train_loss"].append(train_loss)
        history["val_acc"].append(val_acc)
        if (epoch + 1) % 25 == 0:
            print(f"Epoch {epoch+1:03d} | loss {train_loss:.3f} | val acc {val_acc:.3f}")
    return history


## Train: rings


In [ ]:
Xr, yr = make_rings(n_per_class=700, noise=0.12)
train_r, val_r = split_loaders(Xr, yr, batch_size=128, val_ratio=0.2)

model_r = SimpleMLP(width=64)
hist_r = train(model_r, train_r, val_r, epochs=220, lr=1.5e-3)
max(hist_r["val_acc"])


## Train: spiral


In [ ]:
Xs, ys = make_spiral(n_per_class=800, turns=3.0, noise=0.25)
train_s, val_s = split_loaders(Xs, ys, batch_size=128, val_ratio=0.2)

model_s = SimpleMLP(width=96)
hist_s = train(model_s, train_s, val_s, epochs=260, lr=1e-3)
max(hist_s["val_acc"])


## Notes
- Keep it simple: one can start with a small MLP and adjust width/epochs.
- Slightly more epochs for the spiral help it converge.
- Adam is a good default; learning rate around 1e-3 is usually fine.
- Reported accuracy is from a held-out validation split.
